In [1]:
!pip install azure-storage-blob
!pip install pyarrow
!pip install psycopg2 sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.4 MB/s eta 0:00:00


In [2]:
# import libraries
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [3]:
# Function
def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    blob_content = download_stream.readall().decode('utf-8')
    return blob_content

### Download Bronx DataFrame from Azure

In [7]:
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
#Connection_STRING = config["connectionString"]

connection_string = config["connectionString"]
container_name = "groupproject"
blob_name = "groupdata4_Merge_df_Bronx.csv"

blob_content = azure_download_blob(connection_string, container_name, blob_name)
df_Bronx = pd.read_csv(StringIO(blob_content))
df_Bronx

,dba,boro,building,street,zipcode,phone,inspection_date,critical_flag,cuisine_description,action,score,inspection_type,violation_code,violation_description,grade,grade_date,latitude,longitude,yelp_rating,yelp_review_count
0,CORKY'S DINER,Bronx,2535,GRAND CONCOURSE,10468.0,7189332484,2024-01-24,Critical,American,Violations were cited in the following area(s).,18.0,Cycle Inspection / Initial Inspection,02B,Hot TCS food item not held at or above 140 °F.,NaN,NaN,40.863278,-73.896514,3.0,113.0
1,PAPA JOHN'S (STAND 310),Bronx,1,EAST 161 STREET,10451.0,9172843260,2017-07-25,Not Applicable,Pizza,No violations were recorded at the time of thi...,0.0,Cycle Inspection / Initial Inspection,NaN,NaN,A,2017-07-25,40.829028,-73.928496,1.9,24.0
2,JADE PALACE,Bronx,163,EINSTEIN LOOP,10475.0,7183201584,2022-03-23,Critical,Chinese,Violations were cited in the following area(s).,31.0,Cycle Inspection / Initial Inspection,02B,Hot food item not held at or above 140º F.,NaN,NaN,40.864063,-73.822546,2.6,11.0
3,PINE BAR & GRILL,Bronx,1634,EASTCHESTER ROAD,10461.0,7183190900,2017-10-12,Critical,Italian,Violations were cited in the following area(s).,12.0,Cycle Inspection / Re-inspection,06F,Wiping cloths soiled or not stored in sanitizi...,A,2017-10-12,40.845277,-73.845095,3.0,2.0
4,LA ROLA RESTAURANT,Bronx,400,EAST 198 STREET,10458.0,9176881449,2024-03-06,Not Applicable,Spanish,Establishment re-opened by DOHMH.,0.0,Cycle Inspection / Reopening Inspection,NaN,NaN,Z,2024-03-06,40.866021,-73.886021,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12581,XIN HI CHINESE BUFFETT,Bronx,2053,BARTOW AVENUE,10475.0,7183792200,2023-01-20,Not Critical,Chinese,Violations were cited in the following area(s).,5.0,Cycle Inspection / Re-inspection,28-06,Contract with a pest management professional n...,A,2023-01-20,40.869468,-73.828515,4.0,5.0
12582,PROSPECT COFFEE SHOP,Bronx,1309,PROSPECT AVENUE,10459.0,7189911249,2023-03-20,Not Critical,American,Violations were cited in the following area(s).,9.0,Cycle Inspection / Initial Inspection,10F,Non-food contact surface or equipment made of ...,A,2023-03-20,40.828042,-73.898148,0.0,0.0
12583,"SAKE II JAPANESE RESTAURANT, SUSHI & HIBACHI",Bronx,690,EAST 187 STREET,10458.0,7182200988,2023-01-11,Critical,Japanese,Violations were cited in the following area(s).,30.0,Cycle Inspection / Re-inspection,04H,"Raw, cooked or prepared food is adulterated, c...",C,2023-01-11,40.854192,-73.884643,2.7,3.0
12584,FOO-HING KITCHEN,Bronx,2895,SEDGWICK AVENUE,10468.0,7188846267,2021-09-07,Critical,Chinese,Violations were cited in the following area(s).,22.0,Cycle Inspection / Initial Inspection,06B,"Tobacco use, eating, or drinking from open con...",NaN,NaN,40.874944,-73.901262,0.0,0.0


### Create Date Dimension

In [8]:
# Functions
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))

def get_week_of_year(date_str):
    """
    Calculate the ISO week number of the year for a given date.

    Parameters:
    date_str (str): A date string in the format 'YYYY-MM-DD'.

    Returns:
    int: ISO week number of the year.
    """
    # Parse the input string to a datetime object
    date = datetime.strptime(date_str, '%Y-%m-%d')

    # Get the ISO calendar week number
    week_of_year = date.isocalendar()[1]

    return week_of_year

def week_of_month(dt):
    year = dt.year
    month = dt.month
    day = dt.day

    cal = calendar.monthcalendar(year, month)
    week_number = (day - 1) // 7 + 1
    return week_number

In [9]:
min_grade_date = df_Bronx['grade_date'][~pd.isna(df_Bronx['grade_date'])].min()
max_grade_date = df_Bronx['grade_date'][~pd.isna(df_Bronx['grade_date'])].max()

In [10]:
start_date = min(df_Bronx['inspection_date'].min(), min_grade_date)
end_date = max(df_Bronx['inspection_date'].max(), max_grade_date)
print(start_date, end_date)

2016-05-25 2024-04-13


In [11]:
dim_date_df = pd.DataFrame({'date': pd.date_range(start_date, end_date, freq='D')})
dim_date_df.head(10)

,date
0,2016-05-25
1,2016-05-26
2,2016-05-27
3,2016-05-28
4,2016-05-29
5,2016-05-30
6,2016-05-31
7,2016-06-01
8,2016-06-02
9,2016-06-03


In [12]:

# Extract attributes
dim_date_df['Date_ID'] = dim_date_df['date'].dt.strftime('%Y%m%d')
dim_date_df['Date_Iso_Fromat'] = dim_date_df['date'].apply(lambda x: x.isoformat()[:10])

dim_date_df['Year_Number'] = dim_date_df['date'].dt.year
dim_date_df['Quarter_Number'] = dim_date_df['date'].dt.quarter
dim_date_df['Month_Number'] = dim_date_df['date'].dt.month
dim_date_df['Day_Number'] = dim_date_df['date'].dt.day

dim_date_df['Month_Name'] = dim_date_df['date'].dt.strftime('%B')
dim_date_df['Day_Name'] = dim_date_df['date'].dt.strftime('%A')

# Add week of the month and week of the year
dim_date_df['Week_of_the_Year'] = dim_date_df['date'].dt.strftime('%U')
dim_date_df['Week_of_the_Month'] = dim_date_df['date'].apply(week_of_month)

dim_date_df.head(20)

,date,Date_ID,Date_Iso_Fromat,Year_Number,Quarter_Number,Month_Number,Day_Number,Month_Name,Day_Name,Week_of_the_Year,Week_of_the_Month
0,2016-05-25,20160525,2016-05-25,2016,2,5,25,May,Wednesday,21,4
1,2016-05-26,20160526,2016-05-26,2016,2,5,26,May,Thursday,21,4
2,2016-05-27,20160527,2016-05-27,2016,2,5,27,May,Friday,21,4
3,2016-05-28,20160528,2016-05-28,2016,2,5,28,May,Saturday,21,4
4,2016-05-29,20160529,2016-05-29,2016,2,5,29,May,Sunday,22,5
5,2016-05-30,20160530,2016-05-30,2016,2,5,30,May,Monday,22,5
6,2016-05-31,20160531,2016-05-31,2016,2,5,31,May,Tuesday,22,5
7,2016-06-01,20160601,2016-06-01,2016,2,6,1,June,Wednesday,22,1
8,2016-06-02,20160602,2016-06-02,2016,2,6,2,June,Thursday,22,1
9,2016-06-03,20160603,2016-06-03,2016,2,6,3,June,Friday,22,1


In [13]:
# delete 'date' column in dim_date as final date dimension table
dim_date_final_df = dim_date_df.drop(dim_date_df.columns[0], axis=1)
dim_date_final_df

,Date_ID,Date_Iso_Fromat,Year_Number,Quarter_Number,Month_Number,Day_Number,Month_Name,Day_Name,Week_of_the_Year,Week_of_the_Month
0,20160525,2016-05-25,2016,2,5,25,May,Wednesday,21,4
1,20160526,2016-05-26,2016,2,5,26,May,Thursday,21,4
2,20160527,2016-05-27,2016,2,5,27,May,Friday,21,4
3,20160528,2016-05-28,2016,2,5,28,May,Saturday,21,4
4,20160529,2016-05-29,2016,2,5,29,May,Sunday,22,5
...,...,...,...,...,...,...,...,...,...,...
2876,20240409,2024-04-09,2024,2,4,9,April,Tuesday,14,2
2877,20240410,2024-04-10,2024,2,4,10,April,Wednesday,14,2
2878,20240411,2024-04-11,2024,2,4,11,April,Thursday,14,2
2879,20240412,2024-04-12,2024,2,4,12,April,Friday,14,2


# Create Facts Dimension

In [14]:
# Azure Functions
def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [15]:
# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
#Connection_STRING = config["connectionString"]

connection_string = config["connectionString"]
container_name = "groupproject"
blob_name = "groupdata4_Merge_df_Bronx.csv"

blob_content = azure_download_blob(connection_string, container_name, blob_name)
blob_content = blob_content.decode('utf-8')
group_df = pd.read_csv(StringIO(blob_content))
group_df.head()


,dba,boro,building,street,zipcode,phone,inspection_date,critical_flag,cuisine_description,action,score,inspection_type,violation_code,violation_description,grade,grade_date,latitude,longitude,yelp_rating,yelp_review_count
0,CORKY'S DINER,Bronx,2535,GRAND CONCOURSE,10468.0,7189332484,2024-01-24,Critical,American,Violations were cited in the following area(s).,18.0,Cycle Inspection / Initial Inspection,02B,Hot TCS food item not held at or above 140 °F.,NaN,NaN,40.863278,-73.896514,3.0,113.0
1,PAPA JOHN'S (STAND 310),Bronx,1,EAST 161 STREET,10451.0,9172843260,2017-07-25,Not Applicable,Pizza,No violations were recorded at the time of thi...,0.0,Cycle Inspection / Initial Inspection,NaN,NaN,A,2017-07-25,40.829028,-73.928496,1.9,24.0
2,JADE PALACE,Bronx,163,EINSTEIN LOOP,10475.0,7183201584,2022-03-23,Critical,Chinese,Violations were cited in the following area(s).,31.0,Cycle Inspection / Initial Inspection,02B,Hot food item not held at or above 140º F.,NaN,NaN,40.864063,-73.822546,2.6,11.0
3,PINE BAR & GRILL,Bronx,1634,EASTCHESTER ROAD,10461.0,7183190900,2017-10-12,Critical,Italian,Violations were cited in the following area(s).,12.0,Cycle Inspection / Re-inspection,06F,Wiping cloths soiled or not stored in sanitizi...,A,2017-10-12,40.845277,-73.845095,3.0,2.0
4,LA ROLA RESTAURANT,Bronx,400,EAST 198 STREET,10458.0,9176881449,2024-03-06,Not Applicable,Spanish,Establishment re-opened by DOHMH.,0.0,Cycle Inspection / Reopening Inspection,NaN,NaN,Z,2024-03-06,40.866021,-73.886021,0.0,0.0


In [16]:
# Create Facts Dimension
facts_df = group_df[['score', 'yelp_rating', 'yelp_review_count', 'critical_flag','grade']]
facts_df['Fact_ID'] = range(1, len(facts_df) + 1)
facts_dimension = facts_df[['Fact_ID','score', 'yelp_rating', 'yelp_review_count', 'critical_flag','grade']]

facts_dimension.head()

<ipython-input-16-ba52ddd649a3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  facts_df['Fact_ID'] = range(1, len(facts_df) + 1)


,Fact_ID,score,yelp_rating,yelp_review_count,critical_flag,grade
0,1,18.0,3.0,113.0,Critical,NaN
1,2,0.0,1.9,24.0,Not Applicable,A
2,3,31.0,2.6,11.0,Critical,NaN
3,4,12.0,3.0,2.0,Critical,A
4,5,0.0,0.0,0.0,Not Applicable,Z


In [17]:
rename_title = {
    'score': 'Inspection_Score',
    'yelp_rating': 'Yelp_Star',
    'yelp_review_count': 'Yelp_Review_Count',
    'critical_flag': 'Critical_Flag',
    'grade': 'Grage'
}
facts_dimension = facts_dimension.rename(columns=rename_title)
facts_dimension.head()

,Fact_ID,Inspection_Score,Yelp_Star,Yelp_Review_Count,Critical_Flag,Grage
0,1,18.0,3.0,113.0,Critical,NaN
1,2,0.0,1.9,24.0,Not Applicable,A
2,3,31.0,2.6,11.0,Critical,NaN
3,4,12.0,3.0,2.0,Critical,A
4,5,0.0,0.0,0.0,Not Applicable,Z


# Create Location Dimension

In [18]:
# Azure Functions

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()


In [19]:
# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
#Connection_STRING = config["connectionString"]

connection_string = config["connectionString"]
container_name = "groupproject"
blob_name = "groupdata4_Merge_df_Bronx.csv"

blob_content = azure_download_blob(connection_string, container_name, blob_name)
blob_content = blob_content.decode('utf-8')
group_df = pd.read_csv(StringIO(blob_content))
group_df.head()

,dba,boro,building,street,zipcode,phone,inspection_date,critical_flag,cuisine_description,action,score,inspection_type,violation_code,violation_description,grade,grade_date,latitude,longitude,yelp_rating,yelp_review_count
0,CORKY'S DINER,Bronx,2535,GRAND CONCOURSE,10468.0,7189332484,2024-01-24,Critical,American,Violations were cited in the following area(s).,18.0,Cycle Inspection / Initial Inspection,02B,Hot TCS food item not held at or above 140 °F.,NaN,NaN,40.863278,-73.896514,3.0,113.0
1,PAPA JOHN'S (STAND 310),Bronx,1,EAST 161 STREET,10451.0,9172843260,2017-07-25,Not Applicable,Pizza,No violations were recorded at the time of thi...,0.0,Cycle Inspection / Initial Inspection,NaN,NaN,A,2017-07-25,40.829028,-73.928496,1.9,24.0
2,JADE PALACE,Bronx,163,EINSTEIN LOOP,10475.0,7183201584,2022-03-23,Critical,Chinese,Violations were cited in the following area(s).,31.0,Cycle Inspection / Initial Inspection,02B,Hot food item not held at or above 140º F.,NaN,NaN,40.864063,-73.822546,2.6,11.0
3,PINE BAR & GRILL,Bronx,1634,EASTCHESTER ROAD,10461.0,7183190900,2017-10-12,Critical,Italian,Violations were cited in the following area(s).,12.0,Cycle Inspection / Re-inspection,06F,Wiping cloths soiled or not stored in sanitizi...,A,2017-10-12,40.845277,-73.845095,3.0,2.0
4,LA ROLA RESTAURANT,Bronx,400,EAST 198 STREET,10458.0,9176881449,2024-03-06,Not Applicable,Spanish,Establishment re-opened by DOHMH.,0.0,Cycle Inspection / Reopening Inspection,NaN,NaN,Z,2024-03-06,40.866021,-73.886021,0.0,0.0


In [20]:
group_df.columns

Index(['dba', 'boro', 'building', 'street', 'zipcode', 'phone',
       'inspection_date', 'critical_flag', 'cuisine_description', 'action',
       'score', 'inspection_type', 'violation_code', 'violation_description',
       'grade', 'grade_date', 'latitude', 'longitude', 'yelp_rating',
       'yelp_review_count'],
      dtype='object')

In [21]:
#Consolidation
group_df['zipcode'] =group_df['zipcode'].astype('Int64')

In [22]:
# Create Location Dimension
location_df = group_df[['boro', 'building', 'street', 'zipcode','latitude','longitude']].drop_duplicates()
location_df['location_id'] = range(1, len(location_df) + 1)
location_dimension = location_df[['location_id','boro', 'building', 'street', 'zipcode','latitude','longitude']]

location_dimension.head()

,location_id,boro,building,street,zipcode,latitude,longitude
0,1,Bronx,2535,GRAND CONCOURSE,10468,40.863278,-73.896514
1,2,Bronx,1,EAST 161 STREET,10451,40.829028,-73.928496
2,3,Bronx,163,EINSTEIN LOOP,10475,40.864063,-73.822546
3,4,Bronx,1634,EASTCHESTER ROAD,10461,40.845277,-73.845095
4,5,Bronx,400,EAST 198 STREET,10458,40.866021,-73.886021


In [23]:
#Rename columns
new_column_names = {
    'boro': 'Borough',
    'building': 'Building',
    'street':'Street',
    'zipcode':'Zipcode',
    'location_id':'Location_ID',
    'latitude':"Latitude",
    'longitude':'Longitude'

}

location_dimension = location_dimension.rename(columns=new_column_names)
location_dimension

,Location_ID,Borough,Building,Street,Zipcode,Latitude,Longitude
0,1,Bronx,2535,GRAND CONCOURSE,10468,40.863278,-73.896514
1,2,Bronx,1,EAST 161 STREET,10451,40.829028,-73.928496
2,3,Bronx,163,EINSTEIN LOOP,10475,40.864063,-73.822546
3,4,Bronx,1634,EASTCHESTER ROAD,10461,40.845277,-73.845095
4,5,Bronx,400,EAST 198 STREET,10458,40.866021,-73.886021
...,...,...,...,...,...,...,...
11097,1331,Bronx,1544,BOONE AVENUE,10460,40.832998,-73.885099
11750,1332,Bronx,1883,WEBSTER AVENUE,10457,40.847327,-73.901169
11813,1333,Bronx,2559,BOSTON ROAD,10467,40.864550,-73.863699
11824,1334,Bronx,4277,KATONAH AVENUE,10470,40.897845,-73.867251


# Create Violation Dimension

In [24]:
# Azure Functions
def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [25]:
# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
#Connection_STRING = config["connectionString"]

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'groupproject'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)

blob_name = "groupdata4_Merge_df_Bronx.csv"

blob_client = container_client.get_blob_client(blob=blob_name)
blob_data = blob_client.download_blob()
blob_content = blob_data.readall().decode('utf-8')

df = pd.read_csv(StringIO(blob_content))

dim_df = df.copy()

In [26]:
dim_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12586 entries, 0 to 12585
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dba                    12586 non-null  object 
 1   boro                   12586 non-null  object 
 2   building               12586 non-null  object 
 3   street                 12586 non-null  object 
 4   zipcode                12580 non-null  float64
 5   phone                  12586 non-null  object 
 6   inspection_date        12586 non-null  object 
 7   critical_flag          12586 non-null  object 
 8   cuisine_description    12586 non-null  object 
 9   action                 12586 non-null  object 
 10  score                  11988 non-null  float64
 11  inspection_type        12586 non-null  object 
 12  violation_code         12506 non-null  object 
 13  violation_description  12506 non-null  object 
 14  grade                  5972 non-null   object 
 15  gr

In [27]:
# Create Violation Dimension
healthcode_mapping = {
    '02A':'Food not cooked to required minimum internal temperature','02B':'Hot TCS food item not held at or above 140º F.','02C':'Hot TCS food item that has been cooked and cooled is being held for service without first being reheated to 165º F or above for 15 seconds within 2 hours.','02D':'Precooked TCS food in hermetically sealed and intact packages from commercial food processing and non-retail processing establishments not heated to 140º F within 2 hours.','02E':'Whole frozen poultry or poultry breasts, other than a single portion, cooked frozen or partially thawed.',
    '02F':'Meat, fish, poultry, eggs or molluscan shellfish served or offered raw or undercooked without written consumer advisory.','02G':'Cold TCS food item held above 41ºF; smoked or processed fish held above 38° F; intact raw eggs held above 45° F; or reduced oxygen packaged (ROP) TCS foods held above required temperatures except during active necessary preparation.','02H':'After cooking or removal from hot holding, TCS food not cooled by an approved method whereby the internal product temperature is reduced from 140º F to 70º F or less within 2 hours, and from 70º F to 41º F or less within 4 additional hours.','02I':'TCS food removed from cold holding or prepared from or combined with ingredients at room temperature not cooled by an approved method to 41º F or below within 4 additional hours.','02J':'ROP TCS foods not cooled by an approved method as specified in approved HACCPplan.',
    '03A':'Food from unapproved or unknown source or home canned or home prepared. Live animal slaughtered in establishment. ROP fish not frozen before processing; or ROP foods prepared on premises transported to another site.','03B':'Shellfish not from approved source, not or improperly tagged/labeled; tags not retained for 90 days.','03C':'Unclean or cracked whole eggs or unpasteurized liquid, frozen or powdered eggs kept or used.','03D':'Food packages, canned food, hermetically sealed containers swollen, leaking or rusted, or otherwise damaged, without “Do Not Use” label and not segregated from other consumable food items. ',
    '03E':'No or inadequate potable water supply. Water or ice not potable or from unapproved source. Bottled water not NY State certified. Cross connection in potable water supply system.','03F':'Unpasteurized milk or milk product (except certain aged cheese) served.','03G':'Raw fruit or vegetables not properly washed prior to cutting or serving.','03I':'Unpasteurized juice packaged and sealed on premises not labeled or label incomplete; no warning statement.',
    '04A':'Food Protection Certificate (FPC) not held by manager or supervisor of food operations.','04B':'Food worker spits; prepares food or touches utensil when ill with a disease transmissible by food or has exposed infected cut or burn on hand.','04C':'Food worker does not use utensil or other barrier to eliminate bare hand contact with food that will not receive adequate additional heat treatment.','04D':'Food worker does not wash hands thoroughly after using the toilet, coughing, sneezing, smoking, eating, preparing raw foods or otherwise contaminating hands or does not change gloves when gloves are contaminated.','04E':'Toxic chemicals or pesticides improperly labeled, stored or used such that food contamination may occur.','04F':'Food preparation area, food storage area, or other area used by employees or patrons, contaminated by sewage or liquid waste.','04G':'Unprotected TCS food re-served.','04H':'Raw, cooked or prepared food is adulterated, contaminated, cross-contaminated, or not discarded in accordance with HACCP plan.',
    '04I':'Unprotected non-TCS food re-served.','04J':'Properly calibrated thermometer or thermocouple not provided or not readily accessible in food preparation and hot/cold holding areas to measure temperatures of TCS foods during cooking, cooling, reheating, and holding. ','04K':'Evidence of rats or live rats in establishment’s food or non-food areas. ','04L':'Evidence of mice or live mice in establishment’s food or non-food areas.','04M':'Live roaches in establishment’s food or non-food areas. ','04N':'Filth flies or food/refuse/sewage associated (FRSA) flies or other nuisance pests in establishment’s food and/or non-food areas. FRSA flies include house flies, blow flies, bottle flies, flesh flies, drain flies, Phorid flies and fruit flies.','04O':'Live animals other than fish in tank or service animal in establishment’s food or non-food areas.','04P':'Food containing a prohibited substance held, kept, offered, prepared, processed, packaged, or served.',
    '05A':'Sewage disposal system not provided, improper, inadequate or unapproved.','05B':'Harmful, noxious gas or vapor detected. Carbon monoxide (CO) level at or exceeding nine (9) ppm. ','05C':'Food contact surface, refillable, reusable containers, or equipment improperly constructed placed or maintained. Unacceptable material used. Culinary sink or other acceptable method not provided for washing food. ','05D':'No hand wash facility in or adjacent to toilet rooms or within 25 feet of food preparation or service or ware washing area; or hand wash facility not accessible, obstructed or used for non-hand washing purposes. No hot and cold running water or water at inadequate pressure. No soap or acceptable hand-drying device.','05E':'Toilet facility not provided for employees or for patrons when required. Shared patronemployee toilet accessed through kitchen, food prep or storage area or utensil washing area.',
    '05F':'Inadequate or no refrigerated or hot holding equipment to maintain TCS foods at required temperatures. Outdoor refrigerator not locked, secured.','05G':'Separate, enclosed, properly equipped and sized cleaning and service area not provided in mobile food commissary; street, sidewalk used for cleaning units. ','05H':'No facilities available to wash, rinse, and sanitize dishware, glassware, utensils, equipment or refillable returnable containers. No approved written standard operating procedure for avoiding contamination by refillable returnable containers.','05I':'Refrigeration used to implement HACCP plan for ROP foods not equipped with an electronic system that continuously monitors time and temperature.',
    '06A':'Personal cleanliness inadequate. Outer garment soiled with possible contaminant. Effective hair restraint not worn when required. Jewelry worn on hands or arms: fingernail polish worn; or fingernails not kept clean and trimmed.','06B':'Tobacco or e-cigarette use, eating, or drinking from open container in food preparation or other area where food, equipment or utensils may be exposed to contamination.','06C':'Food, supplies, or equipment not protected from potential source of contamination during storage, preparation, transportation, display, service or from customer’s refillable, reusable container.','06D':'Food contact surface not properly washed, rinsed and sanitized after any activity where contamination may have occurred.','06E':'Sanitized equipment or utensil, including in-use food dispensing utensil, improperly used or stored.',
    '06F':'Wiping cloths soiled or not stored in sanitizing solution; inadequately sanitized.','06G':'HACCP plan not approved or approved HACCP plan not maintained on premises.','06H':'Records and logs not maintained to show that approved HACCP plan has been properly implemented. ','06I':'ROP TCS food not labeled in accordance with approved HACCP plan.','06J':'Refillable, reusable beverage container filled with TCS foods.',
    '07A':'Duties of an officer of the Department interfered with or obstructed.',
    '08A':'Harborage or conditions conducive to attracting pests to the premises and/or allowing pests to exist when pests or signs of pests are present.','08B':'Garbage receptacles not pest or water resistant, or covered with tight-fitting lids, except while in active use. Garbage receptacles and covers not cleaned after emptying and prior to reuse.','08C':'Pesticide use not in accordance with label or applicable laws. Pesticides, other toxic chemicals improperly used/stored. Unprotected, unlocked bait station used.',
    '09A':'Cans of food with dented body damage not segregated from other cans for return to distributor.','09B':'Thawing procedures improper.','09C':'Food contact surface chipped, cracked, worn, or in a condition where it cannot be properly maintained or cleaned.','09D':'Food service operation occurring in room or area used as living or sleeping quarters.','09E':'“Wash hands” sign not posted at hand wash facility.',
    '10A':'Toilet facility not maintained or provided with toilet paper, waste receptacle or self-closing door','10B':'Back-siphonage or back-flow prevention device not provided where required; equipment or floor not properly drained; sewage disposal system in disrepair or not functioning properly. Condensation or liquid waste improperly disposed of.','10C':'Lighting inadequate; permanent lighting not provided in food preparation areas, ware washing areas, and storage areas.','10D':'Mechanical or natural ventilation system not provided, improperly installed, in disrepair and/or fails to prevent excessive build-up of grease, heat, steam condensation vapors, odors, smoke, and fumes.','10E':'Accurate thermometer not provided or properly located in refrigerated or hot holding equipment.',
    '10F':'Non-food contact surface or equipment improperly maintained or made of unacceptable material, or not kept clean, or not properly sealed, raised, spaced or movable to allow accessibility for cleaning on all sides, above and underneath equipment or other structures.','10G':'Proper sanitization not used or provided for ware washing operation. No test kit.','10H':'Single service item reused, not protected from contamination when transported, stored, or dispensed; not used when required.','10I':'Single service item reused, improperly stored, dispensed; not used when required.','10J':'“Wash hands” sign not posted at hand-wash facility',
    '99B':'Other general.',
    '15A':'Tobacco vending machine present where prohibited.',
    '15B':'Tobacco vending machine placed less than 25 feet from entrance to premises.',
    '15C':'Tobacco vending machine not visible to the operator, employee or agent.',
    '15D':'Durable sign with license number, expiration date, address and phone number not posted.',
    '15E':'Out-of-package sale of tobacco products observed.',
    '15E2':'Flavored tobacco products sold or offered for sale.',
    '15F':'Out-of-package sale of tobacco products observed.',
    '15F7':'Sign prohibiting smoking or using electronic cigarettes not conspicuously posted.',
    '15G':'Sale of cigarettes, tobacco products, liquid nicotine or electronic cigarettes to persons under age 21 observed.',
    '15H':'Sign prohibiting sale of tobacco products/electronic cigarettes to persons under age 21 not conspicuously posted.',
    '15HH':'Sign prohibiting sale of non-tobacco shisha, pipes, or rolling papers to persons under age 18 not conspicuously posted.',
    '15I':'Cigars that cost more than $3 each not sold in packages of four or more. ',
    '16A':'A food containing artificial trans fat, with 0.5 grams or more of trans fat per serving, is being stored, distributed, held for service, used in preparation of a menu item or served.','16B':'The original nutritional fact labels and/or ingredient label for a cooking oil, shortening or margarine or food item sold in bulk or acceptable manufacturer’s documentation not maintained on site.','16C':'Caloric content not posted on menus, menu boards or food tags, in a food service establishment that is 1 of 15 or more outlets operating the same type of business nationally under common ownership or control, or as a franchise or doing business under the same name, for each menu item that is served in portions, the size and content of which are standardized.','16E':'Caloric content range (minimum to maximum) not posted on menus and or menu boards for each flavor, variety and size of each menu item that is offered for sale in different flavors, varieties and sizes.',
    '16F':'Specific caloric content or range thereof not posted on menus, menu boards or food tags for each menu item offered as a combination meal with multiple options that are listed as single items.','16J':'Sodium warning icon not posted on menus, menu boards or food tags for food items that contain 2,300 mg or more of sodium in a food service establishment that is 1 of 15 or more outlets operating the same type of business nationally under common ownership or control, or as a franchise or doing business under the same name, for each menu item that is served in portions, the size and content of which are standardized.','16K':'Sodium warning icon posted on menus, menu boards or food tags for food items that contain 2,300 mg or more of sodium is not a black and white equilateral triangle; and/or the equilateral triangle is not as wide as it is tall, and/or is not equal in height to the largest letter in the food item’s name, as displayed on the menu, menu board or tag.','16L':'Sodium warning statement not posted conspicuously at the point of purchase. “Warning: [icon image] indicates that the sodium (salt) content of this item is higher than the total daily recommended limit (2,300 mg). High sodium intake can increase blood pressure and risk of heart disease and stroke.”',
    '18A':'Current valid permit, registration or other authorization to operate establishment not available.',
    '18B':'Document issued by the Board of Health, Commissioner or Department unlawfully reproduced or altered.',
    '18C':'Notice of the Department or Board of Health mutilated, obstructed or removed.',
    '18D':'Failure to comply with an Order of the Board of Health, Commissioner or Department.',
    '18E':'Failure to report occurrences of suspected food-borne illness to the Department.',
    '18F':'Permit not conspicuously displayed.',
    '18G':'Food Protection Certificate not available for Department inspection.',
    '18H':'Failure of event sponsor to exclude vendor without a current valid permit or registration. Operator of shared kitchen allowed unpermitted caterer or other user; or failed to provide copy of agreement between operator and user. ',
    '18I':'Failure to produce pest management contract; failure to keep records showing existence of contract at establishment.',
    '18J':'Unapproved outdoor, street or sidewalk cooking.',
    '20A':'Food allergy information poster not conspicuously posted where food is being prepared or processed by food workers.','20B':'Food allergy information poster not posted in language understood by all food workers.','20C':'Food allergy poster does not contain text provided or approved by Department.','20D':'“Choking first aid” poster not posted. “Alcohol and pregnancy” warning sign not posted. “Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves” sign not posted.','20E':'Letter grade or “Grade Pending” card not conspicuously posted and visible to passersby.','20F':'Current letter grade or “Grade Pending” card not posted.','20G':'Food Protection Certificate not available for inspection.',
    '22A':'Nuisance created or allowed to exist. Facility not free from unsafe, hazardous, offensive or annoying conditions.','22C':'Bulb not shielded or shatterproof, and/or endcaps or other devices not provided in areas where there is extreme heat, temperature changes, or where accidental contact may occur.','22D':'Plastic containers with microwave safe markings not used for heating food.','22E':'ROP processing equipment not approved by the Department.','22F':'Misbranded, mislabeled packaged food products.',
    '15-01':'Smoking or electronic cigarette use allowed in prohibited area.',
    '15-21':'Flavored tobacco products sold, offered for sale.',
    '15-22':'Original label for smoking products sold or offered for sale.',
    '15-37':'Workplace SFAA policy not prominently posted in workplace.',
    '15-42':'Failure to make a good faith effort to inform smokers or electronic cigarette users of Smoke-Free Air Act ("SFAA") prohibitions.',
    '16-02':'The original nutrition fact labels or ingredient label for a cooking oil, shortening or margarine or food item sold in bulk, or acceptable manufacturer’s documentation not maintained on site.',
    '16-03':'Caloric content not posted on menus, menu boards or food tags, in a food service establishment that is 1 of 15 or more outlets operating the same type of business nationally under common ownership or control, or as a franchise or doing business under the same name, for each menu item that is served in portions, the size and content of which are standardized.',
    '16-04':'Required succinct nutritional statements not posted on menu(s) for adults and children (2,000 calories per day for adults)',
    '18-01':'Current valid permit, registration or other authorization to operate a Food Service Establishment (FSE) or Non-retail Food Processing Establishment (NRFP) not available.',
    '18-11':'Food Protection Certificate not available for inspection.',
    '19-04':'Expanded Polystyrene (EPS) single service article not designated as a recyclable material.',
    '19-06':'Providing single-use, non-compostable plastic straws to customers without customer request (including providing such straws at a self-serve station).',
    '19-07':'Failure to maintain a sufficient supply of single-use, non-compostable plastic straws.',
    '19-10':'Failure to display required signage about plastic straw availability.',
    '20-01':'Food allergy information poster not conspicuously posted where food is being prepared or processed by food workers.',
    '20-04':'“Choking first aid” poster not posted. “Alcohol and pregnancy” warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted.',
    '20-06':'Current letter grade or Grade Pending card not posted.',
    '20-08':'Failure to post or conspicuously post healthy eating information.',
    '28-01':'Nuisance created or allowed to exist. Facility not free from unsafe, hazardous, offensive or annoying condition.',
    '28-03':'Lighting fixture located over, by or within food storage, preparation, service or display facility, and facility where utensils and equipment are cleaned and stored, which may shatter due to extreme heat, temperature changes or accidental contact; not fitted with shatterproof bulb or shielded and encased, with end caps or other device.',
    '28-04':'Equipment used for ROP not approved by the Department.',
    '28-05':'Food adulterated or misbranded. Adulterated or misbranded food possessed, being manufactured, produced, packed, sold, offered for sale, delivered or given away',
    '28-06':'Contract with a pest management professional not in place. Record of extermination activities not kept on premises.'
    }


In [28]:
codes = list(healthcode_mapping.keys())
descriptions = list(healthcode_mapping.values())

violation_df = pd.DataFrame({'Violation_ID': range(1, len(codes) + 1), 'Violation_Code': codes, 'Violation_Description': descriptions})
violation_df.head()

,Violation_ID,Violation_Code,Violation_Description
0,1,02A,Food not cooked to required minimum internal t...
1,2,02B,Hot TCS food item not held at or above 140º F.
2,3,02C,Hot TCS food item that has been cooked and coo...
3,4,02D,Precooked TCS food in hermetically sealed and ...
4,5,02E,"Whole frozen poultry or poultry breasts, other..."


# Create Restaurant Dimension

In [29]:
#Create Restaurant Dimension
restaurant_df = dim_df.loc[:, ["dba", "phone", "cuisine_description"]]
restaurant_df['Restaurant_ID'] = range(1, len(restaurant_df) + 1)
rename_mapping = {
    'dba': 'Restaurant_Name',  # Similarly, adjust according to actual intended names
    'phone': 'Phone',
    'cuisine_description': 'Cuisine_Description',
}

restaurant_df = restaurant_df.rename(columns=rename_mapping)
new_order = ['Restaurant_ID', 'Restaurant_Name','Phone','Cuisine_Description']
restaurant_df = restaurant_df[new_order]

restaurant_df.head()

,Restaurant_ID,Restaurant_Name,Phone,Cuisine_Description
0,1,CORKY'S DINER,7189332484,American
1,2,PAPA JOHN'S (STAND 310),9172843260,Pizza
2,3,JADE PALACE,7183201584,Chinese
3,4,PINE BAR & GRILL,7183190900,Italian
4,5,LA ROLA RESTAURANT,9176881449,Spanish


# Create Inspection Type Dimension

In [30]:
#Create Inspection Type Dimension
inspection_types = {
    1:'Inter-Agency Task Force / Initial Inspection',
    2:'Cycle Inspection / Initial Inspection',
    3:'Administrative Miscellaneous / Re-inspection',
    4:'Administrative Miscellaneous / Initial Inspection',
    5:'Trans Fat / Initial Inspection',
    6:'Cycle Inspection / Reopening Inspection',
    7:'Cycle Inspection / Re-inspection',
    8:'Smoke-Free Air Act / Limited Inspection',
    9:'Smoke-Free Air Act / Initial Inspection',
    10:'Pre-permit (Operational) / Re-inspection',
    11:'Pre-permit (Operational) / Initial Inspection',
    12:'Calorie Posting / Initial Inspection',
    13:'Pre-permit (Operational) / Reopening Inspection',
    14:'Pre-permit (Non-operational) / Initial Inspection',
    15:'Pre-permit (Non-operational) / Re-inspection',
    16:'Cycle Inspection / Compliance Inspection',
    17:'Pre-permit (Operational) / Compliance Inspection',
    18:'Pre-permit (Operational) / Second Compliance Inspection',
    19:'Cycle Inspection / Second Compliance Inspection',
    20:'Smoke-Free Air Act / Re-inspection',
    21:'Calorie Posting / Re-inspection',
    22:'Calorie Posting / Compliance Inspection',
    23:'Administrative Miscellaneous / Reopening Inspection',
    24:'Administrative Miscellaneous / Compliance Inspection',
    25:'Pre-permit (Non-operational) / Compliance Inspection',
    26:'Trans Fat / Compliance Inspection',
    27:'Administrative Miscellaneous / Second Compliance Inspection',
    28:'Smoke-Free Air Act / Compliance Inspection',
    29:'Pre-permit (Non-operational) / Second Compliance Inspection',
    30:'Inter-Agency Task Force / Re-inspection',
    31:'Trans Fat / Second Compliance Inspection',
    32:'Trans Fat / Re-inspection'
}

inspection_series = pd.Series(inspection_types)

inspection_ids, unique_inspection_types = pd.factorize(inspection_series)
inspection_ids += 1

inspectiontype_df = pd.DataFrame({'Inspection_ID': inspection_ids, 'Inspection_Type': inspection_series})

inspectiontype_df.head(5)

,Inspection_ID,Inspection_Type
1,1,Inter-Agency Task Force / Initial Inspection
2,2,Cycle Inspection / Initial Inspection
3,3,Administrative Miscellaneous / Re-inspection
4,4,Administrative Miscellaneous / Initial Inspection
5,5,Trans Fat / Initial Inspection


## Merge Data to Facts Table



In [38]:
merged_df = pd.concat([facts_dimension, dim_date_final_df['Date_ID'],
                       location_dimension['Location_ID'], violation_df['Violation_ID'],
                       restaurant_df['Restaurant_ID'], inspectiontype_df['Inspection_ID']], axis=1)

In [39]:
merged_df

,Fact_ID,Inspection_Score,Yelp_Star,Yelp_Review_Count,Critical_Flag,Grage,Date_ID,Location_ID,Violation_ID,Restaurant_ID,Inspection_ID
0,1,18.0,3.0,113.0,Critical,NaN,20160525,1.0,1.0,1,NaN
1,2,0.0,1.9,24.0,Not Applicable,A,20160526,2.0,2.0,2,1.0
2,3,31.0,2.6,11.0,Critical,NaN,20160527,3.0,3.0,3,2.0
3,4,12.0,3.0,2.0,Critical,A,20160528,4.0,4.0,4,3.0
4,5,0.0,0.0,0.0,Not Applicable,Z,20160529,5.0,5.0,5,4.0
...,...,...,...,...,...,...,...,...,...,...,...
12581,12582,5.0,4.0,5.0,Not Critical,A,NaN,NaN,NaN,12582,NaN
12582,12583,9.0,0.0,0.0,Not Critical,A,NaN,NaN,NaN,12583,NaN
12583,12584,30.0,2.7,3.0,Critical,C,NaN,NaN,NaN,12584,NaN
12584,12585,22.0,0.0,0.0,Critical,NaN,NaN,NaN,NaN,12585,NaN
